In [1]:
from nltk.translate.bleu_score import corpus_bleu
from keras.models import Model,load_model
from keras.callbacks import ModelCheckpoint
from keras.engine.topology import Input
from keras.layers.core import Dense,Dropout
from keras.layers.recurrent import LSTM
from keras.layers.embeddings import Embedding
from keras.layers.merge import add
from keras.regularizers import l2
from pickle import dump ,load
from keras.utils import plot_model
from keras.preprocessing.sequence import pad_sequences
import numpy as np
import os
os.chdir('/home/aninda/Desktop/testProject/captionGenerator/')

/home/aninda/anaconda3/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
embedding_weight=load(open('embedding_weight.pkl','rb'))
vocab_size=2936
max_len=34
regularizer=l2(0)

In [8]:
photo_input=Input(shape=(4096,),name='photo_input')
photo_drop=Dropout(.5,name='photo_drop')(photo_input)
photo_dence=Dense(256,activation='relu',name='photo_dence')(photo_drop)

text_input=Input(shape=(max_len,),name='text_input')
text_emb=Embedding(vocab_size+1,100,weights=[embedding_weight],embeddings_regularizer=regularizer, mask_zero=True,name='text_emb')(text_input)
text_drop=Dropout(.5,name='text_drop')(text_emb)
text_lstm=LSTM(256,name='text_lstm')(text_drop)

decoder_add=add([photo_dence,text_lstm],name='decoder_add')
decoder_dence=Dense(256,activation='relu',name='decoder_dence')(decoder_add)
output_dence=Dense(vocab_size+1,activation='softmax',name='output_dence')(decoder_dence)

model=Model(input=[photo_input,text_input],outputs=output_dence)
model.compile(loss='categorical_crossentropy', optimizer='adam')
print(model.summary())

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
text_input (InputLayer)         (None, 34)           0                                            
__________________________________________________________________________________________________
photo_input (InputLayer)        (None, 4096)         0                                            
__________________________________________________________________________________________________
text_emb (Embedding)            (None, 34, 100)      293700      text_input[0][0]                 
__________________________________________________________________________________________________
photo_drop (Dropout)            (None, 4096)         0           photo_input[0][0]                
__________________________________________________________________________________________________
text_drop 

/home/aninda/anaconda3/lib/python3.5/site-packages/ipykernel_launcher.py:14: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("ou..., inputs=[<tf.Tenso...)`
  


In [11]:
plot_model(model,to_file='model.jpg',show_shapes=True)

In [5]:
x1_train=load(open('x1_train.pkl','rb'))
x2_train=load(open('x2_train.pkl','rb'))
y_train=load(open('y_train.pkl','rb'))

In [6]:
x1_val=load(open('x1_val.pkl','rb'))
x2_val=load(open('x2_val.pkl','rb'))
y_val=load(open('y_val.pkl','rb'))

In [9]:
filepath = 'model/model-ep{epoch:03d}-loss{loss:.3f}-val_loss{val_loss:.3f}.h5'
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
model.fit([x1_train, x2_train], y_train, epochs=10, verbose=2, batch_size=500,callbacks=[checkpoint], validation_data=([x1_val, x2_val], y_val))

Train on 60638 samples, validate on 9860 samples
Epoch 1/10
 - 231s - loss: 5.2674 - val_loss: 4.5979

Epoch 00001: val_loss improved from inf to 4.59790, saving model to model-ep001-loss5.267-val_loss4.598.h5
Epoch 2/10
 - 229s - loss: 4.3656 - val_loss: 4.0787

Epoch 00002: val_loss improved from 4.59790 to 4.07867, saving model to model-ep002-loss4.366-val_loss4.079.h5
Epoch 3/10
 - 227s - loss: 3.8675 - val_loss: 3.8798

Epoch 00003: val_loss improved from 4.07867 to 3.87977, saving model to model-ep003-loss3.868-val_loss3.880.h5
Epoch 4/10
 - 226s - loss: 3.5318 - val_loss: 3.8184

Epoch 00004: val_loss improved from 3.87977 to 3.81843, saving model to model-ep004-loss3.532-val_loss3.818.h5
Epoch 5/10
 - 228s - loss: 3.2665 - val_loss: 3.8171

Epoch 00005: val_loss improved from 3.81843 to 3.81707, saving model to model-ep005-loss3.266-val_loss3.817.h5
Epoch 6/10
 - 227s - loss: 3.0391 - val_loss: 3.8480

Epoch 00006: val_loss did not improve
Epoch 7/10
 - 226s - loss: 2.8489 - va

In [10]:
test_description_list=load(open('test_description_list.pkl','rb'))
test_image_id=load(open('test_image_id.pkl','rb'))
all_features=load(open('features.pkl','rb'))
tokenizer=load(open('tokenizer.pkl','rb'))
index_word=load(open('index_word.pkl','rb'))
model=load_model('model/model-ep005-loss3.266-val_loss3.817.h5')


In [11]:
def pred_description(model,tokenizer,photo,mx_len=34,):
    in_text='startseq'
    print(type(tokenizer))
    for i in range(mx_len):
        sequence = tokenizer.texts_to_sequences([in_text])[0]
        sequence = pad_sequences([sequence], maxlen=mx_len)
        pred = model.predict([photo,sequence], verbose=0)
        pred=np.argmax(pred)
        pred_word=index_word[pred]
        in_text+=' '+pred_word
        if pred_word=='endseq' or pred_word is None:
            break
    return in_text
 

In [12]:
def evaluate_model(model, descriptions, photos, tokenizer):
    actual, predicted =[],[]
    for photo_id_idx,desc in enumerate(descriptions):
        pred_caption = pred_description(model, tokenizer, photos[test_image_id[photo_id_idx]])
        references = [desc.split()]
        actual.append(references)
        predicted.append(pred_caption.split())
    # calculate BLEU score
    print('BLEU-1: %f' % corpus_bleu(actual, predicted, weights=(1.0, 0, 0, 0)))
    print('BLEU-2: %f' % corpus_bleu(actual, predicted, weights=(0.5, 0.5, 0, 0)))
    print('BLEU-3: %f' % corpus_bleu(actual, predicted, weights=(0.3, 0.3, 0.3, 0)))
    print('BLEU-4: %f' % corpus_bleu(actual, predicted, weights=(0.25, 0.25, 0.25, 0.25)))

In [13]:
evaluate_model(model,test_description_list, all_features, tokenizer)

<class 'keras.preprocessing.text.Tokenizer'>
<class 'keras.preprocessing.text.Tokenizer'>
<class 'keras.preprocessing.text.Tokenizer'>
<class 'keras.preprocessing.text.Tokenizer'>
<class 'keras.preprocessing.text.Tokenizer'>
<class 'keras.preprocessing.text.Tokenizer'>
<class 'keras.preprocessing.text.Tokenizer'>
<class 'keras.preprocessing.text.Tokenizer'>
<class 'keras.preprocessing.text.Tokenizer'>
<class 'keras.preprocessing.text.Tokenizer'>
<class 'keras.preprocessing.text.Tokenizer'>
<class 'keras.preprocessing.text.Tokenizer'>
<class 'keras.preprocessing.text.Tokenizer'>
<class 'keras.preprocessing.text.Tokenizer'>
<class 'keras.preprocessing.text.Tokenizer'>
<class 'keras.preprocessing.text.Tokenizer'>
<class 'keras.preprocessing.text.Tokenizer'>
<class 'keras.preprocessing.text.Tokenizer'>
<class 'keras.preprocessing.text.Tokenizer'>
<class 'keras.preprocessing.text.Tokenizer'>
<class 'keras.preprocessing.text.Tokenizer'>
<class 'keras.preprocessing.text.Tokenizer'>
<class 'ke

<class 'keras.preprocessing.text.Tokenizer'>
<class 'keras.preprocessing.text.Tokenizer'>
<class 'keras.preprocessing.text.Tokenizer'>
<class 'keras.preprocessing.text.Tokenizer'>
<class 'keras.preprocessing.text.Tokenizer'>
<class 'keras.preprocessing.text.Tokenizer'>
<class 'keras.preprocessing.text.Tokenizer'>
<class 'keras.preprocessing.text.Tokenizer'>
<class 'keras.preprocessing.text.Tokenizer'>
<class 'keras.preprocessing.text.Tokenizer'>
<class 'keras.preprocessing.text.Tokenizer'>
<class 'keras.preprocessing.text.Tokenizer'>
<class 'keras.preprocessing.text.Tokenizer'>
<class 'keras.preprocessing.text.Tokenizer'>
<class 'keras.preprocessing.text.Tokenizer'>
<class 'keras.preprocessing.text.Tokenizer'>
<class 'keras.preprocessing.text.Tokenizer'>
<class 'keras.preprocessing.text.Tokenizer'>
<class 'keras.preprocessing.text.Tokenizer'>
<class 'keras.preprocessing.text.Tokenizer'>
<class 'keras.preprocessing.text.Tokenizer'>
<class 'keras.preprocessing.text.Tokenizer'>
<class 'ke

<class 'keras.preprocessing.text.Tokenizer'>
<class 'keras.preprocessing.text.Tokenizer'>
<class 'keras.preprocessing.text.Tokenizer'>
<class 'keras.preprocessing.text.Tokenizer'>
<class 'keras.preprocessing.text.Tokenizer'>
<class 'keras.preprocessing.text.Tokenizer'>
<class 'keras.preprocessing.text.Tokenizer'>
<class 'keras.preprocessing.text.Tokenizer'>
<class 'keras.preprocessing.text.Tokenizer'>
<class 'keras.preprocessing.text.Tokenizer'>
<class 'keras.preprocessing.text.Tokenizer'>
<class 'keras.preprocessing.text.Tokenizer'>
<class 'keras.preprocessing.text.Tokenizer'>
<class 'keras.preprocessing.text.Tokenizer'>
<class 'keras.preprocessing.text.Tokenizer'>
<class 'keras.preprocessing.text.Tokenizer'>
<class 'keras.preprocessing.text.Tokenizer'>
<class 'keras.preprocessing.text.Tokenizer'>
<class 'keras.preprocessing.text.Tokenizer'>
<class 'keras.preprocessing.text.Tokenizer'>
<class 'keras.preprocessing.text.Tokenizer'>
<class 'keras.preprocessing.text.Tokenizer'>
<class 'ke

<class 'keras.preprocessing.text.Tokenizer'>
<class 'keras.preprocessing.text.Tokenizer'>
<class 'keras.preprocessing.text.Tokenizer'>
<class 'keras.preprocessing.text.Tokenizer'>
<class 'keras.preprocessing.text.Tokenizer'>
<class 'keras.preprocessing.text.Tokenizer'>
<class 'keras.preprocessing.text.Tokenizer'>
<class 'keras.preprocessing.text.Tokenizer'>
<class 'keras.preprocessing.text.Tokenizer'>
<class 'keras.preprocessing.text.Tokenizer'>
<class 'keras.preprocessing.text.Tokenizer'>
<class 'keras.preprocessing.text.Tokenizer'>
<class 'keras.preprocessing.text.Tokenizer'>
<class 'keras.preprocessing.text.Tokenizer'>
<class 'keras.preprocessing.text.Tokenizer'>
<class 'keras.preprocessing.text.Tokenizer'>
<class 'keras.preprocessing.text.Tokenizer'>
<class 'keras.preprocessing.text.Tokenizer'>
<class 'keras.preprocessing.text.Tokenizer'>
<class 'keras.preprocessing.text.Tokenizer'>
<class 'keras.preprocessing.text.Tokenizer'>
<class 'keras.preprocessing.text.Tokenizer'>
<class 'ke

<class 'keras.preprocessing.text.Tokenizer'>
<class 'keras.preprocessing.text.Tokenizer'>
<class 'keras.preprocessing.text.Tokenizer'>
<class 'keras.preprocessing.text.Tokenizer'>
<class 'keras.preprocessing.text.Tokenizer'>
<class 'keras.preprocessing.text.Tokenizer'>
<class 'keras.preprocessing.text.Tokenizer'>
<class 'keras.preprocessing.text.Tokenizer'>
<class 'keras.preprocessing.text.Tokenizer'>
<class 'keras.preprocessing.text.Tokenizer'>
<class 'keras.preprocessing.text.Tokenizer'>
<class 'keras.preprocessing.text.Tokenizer'>
<class 'keras.preprocessing.text.Tokenizer'>
<class 'keras.preprocessing.text.Tokenizer'>
<class 'keras.preprocessing.text.Tokenizer'>
<class 'keras.preprocessing.text.Tokenizer'>
<class 'keras.preprocessing.text.Tokenizer'>
<class 'keras.preprocessing.text.Tokenizer'>
<class 'keras.preprocessing.text.Tokenizer'>
<class 'keras.preprocessing.text.Tokenizer'>
<class 'keras.preprocessing.text.Tokenizer'>
<class 'keras.preprocessing.text.Tokenizer'>
<class 'ke

<class 'keras.preprocessing.text.Tokenizer'>
<class 'keras.preprocessing.text.Tokenizer'>
<class 'keras.preprocessing.text.Tokenizer'>
<class 'keras.preprocessing.text.Tokenizer'>
<class 'keras.preprocessing.text.Tokenizer'>
<class 'keras.preprocessing.text.Tokenizer'>
<class 'keras.preprocessing.text.Tokenizer'>
<class 'keras.preprocessing.text.Tokenizer'>
<class 'keras.preprocessing.text.Tokenizer'>
<class 'keras.preprocessing.text.Tokenizer'>
<class 'keras.preprocessing.text.Tokenizer'>
<class 'keras.preprocessing.text.Tokenizer'>
<class 'keras.preprocessing.text.Tokenizer'>
<class 'keras.preprocessing.text.Tokenizer'>
<class 'keras.preprocessing.text.Tokenizer'>
<class 'keras.preprocessing.text.Tokenizer'>
<class 'keras.preprocessing.text.Tokenizer'>
<class 'keras.preprocessing.text.Tokenizer'>
<class 'keras.preprocessing.text.Tokenizer'>
<class 'keras.preprocessing.text.Tokenizer'>
<class 'keras.preprocessing.text.Tokenizer'>
<class 'keras.preprocessing.text.Tokenizer'>
<class 'ke